# CSCI 4022 Final Project Data Cleaning



In [1]:
# imports
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from mpl_toolkits.mplot3d import Axes3D
import scipy.stats as stats
from sklearn.mixture import GaussianMixture

## Data Cleaning

The goal of the data cleaning is to get rid of columns with limited to no data along with columns that have unnecessary or irrelevant data.

### CSV Files
- teams.csv: team data
- team_box_scores.csv: team level box score data from the 2013-2014 season up until the 2017-2018 season
- player_box_scores.csv: player level box score data from the 2013-2014 season up until the 2017-2018 season

### teams.csv

The teams.csv data is going to be used to create a dataframe that can be used to easily identify teams. The cleaning process will get rid of irrelevant data.

In [2]:
# read in the teams csv and see the first few rows
dfTeamsUC = pd.read_csv('teams.csv')
dfTeamsUC.head()

,market,alias,name,id,code_ncaa,kaggle_team_id,school_ncaa,turner_name,league_name,league_alias,...,venue_city,venue_state,venue_address,venue_zip,venue_country,venue_name,venue_capacity,logo_large,logo_medium,logo_small
0,Princeton,PRIN,Tigers,fe406882-9f22-495e-9df6-ef357a6803c6,554,1343,Princeton,Princeton University,NCAA MEN,NCAAM,...,Princeton,NJ,Princeton University,8542.0,USA,Jadwin Gymnasium,6854,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...
1,Yale,YALE,Bulldogs,ca478771-aa3d-4231-81e0-b70f519134fb,813,1463,Yale,Yale University,NCAA MEN,NCAAM,...,New Haven,CT,70 Tower Pkwy,6511.0,USA,John J. Lee Amphitheater,2532,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...
2,Harvard,HARV,Crimson,5c7bf63f-bc39-43c5-9907-73b50b7a6b34,275,1217,Harvard,Harvard University,NCAA MEN,NCAAM,...,Allston,MA,Soldiers Field Road,2163.0,USA,Lavietes Pavilion,2195,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...
3,Dartmouth,DART,Big Green,d60357bd-1205-42e9-9092-d986a2843a34,172,1171,Dartmouth,Dartmouth College,NCAA MEN,NCAAM,...,Hanover,NH,6 South Park Street,3755.0,USA,Edward Leede Arena,2100,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...
4,Cornell,COR,Big Red,88ff8c00-958e-4ccf-a21d-77fab9e93692,167,1165,Cornell,Cornell University,NCAA MEN,NCAAM,...,Ithaca,NY,Bartels Hall,14853.0,USA,Newman Arena,4473,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...,https://www.ncaa.com/sites/default/files/image...


In [3]:
# list out all of the columns
dfTeamsUC.columns

Index(['market', 'alias', 'name', 'id', 'code_ncaa', 'kaggle_team_id',
       'school_ncaa', 'turner_name', 'league_name', 'league_alias',
       'league_id', 'conf_name', 'conf_alias', 'conf_id', 'division_name',
       'division_alias', 'division_id', 'venue_id', 'venue_city',
       'venue_state', 'venue_address', 'venue_zip', 'venue_country',
       'venue_name', 'venue_capacity', 'logo_large', 'logo_medium',
       'logo_small'],
      dtype='object')

None of the venue columns will be valuable except the venue capacity if we want to look at home court advatage, so we can drop them. 

We also won't need the links to the team logos. 

Additionally, there are multiple id columns. We will just use the standard id column and can drop the others.

The turner_name seems to be essentially the same as the market column, so we will keep market and drop turner.

In [4]:
dfTeamsUC1 = dfTeamsUC.drop(['venue_id', 'venue_city',
       'venue_state', 'venue_address', 'venue_zip', 'venue_country',
       'venue_name', 'logo_large', 'logo_medium',
       'logo_small', 'code_ncaa', 'kaggle_team_id',
       'school_ncaa','turner_name'], axis=1)

Out of the remaining columns, lets see how many unique values exist in each column

In [5]:
for col in dfTeamsUC1.columns:
    print(col, len(dfTeamsUC1[col].unique()))

market 351
alias 351
name 209
id 351
league_name 1
league_alias 1
league_id 1
conf_name 32
conf_alias 32
conf_id 32
division_name 1
division_alias 1
division_id 1
venue_capacity 255


Since all the teams are in the same league and division, we can drop all of the corresponding columns.

In [6]:
dfTeamsUC2 = dfTeamsUC1.drop(['league_name', 'league_alias',
       'league_id', 'division_name',
       'division_alias', 'division_id'], axis=1)

Now lets take a look at the resulting dataset.

In [7]:
dfTeamsUC2.head()

,market,alias,name,id,conf_name,conf_alias,conf_id,venue_capacity
0,Princeton,PRIN,Tigers,fe406882-9f22-495e-9df6-ef357a6803c6,Ivy,IVY,c2d16439-b791-4f9b-84d9-80cf457a0676,6854
1,Yale,YALE,Bulldogs,ca478771-aa3d-4231-81e0-b70f519134fb,Ivy,IVY,c2d16439-b791-4f9b-84d9-80cf457a0676,2532
2,Harvard,HARV,Crimson,5c7bf63f-bc39-43c5-9907-73b50b7a6b34,Ivy,IVY,c2d16439-b791-4f9b-84d9-80cf457a0676,2195
3,Dartmouth,DART,Big Green,d60357bd-1205-42e9-9092-d986a2843a34,Ivy,IVY,c2d16439-b791-4f9b-84d9-80cf457a0676,2100
4,Cornell,COR,Big Red,88ff8c00-958e-4ccf-a21d-77fab9e93692,Ivy,IVY,c2d16439-b791-4f9b-84d9-80cf457a0676,4473


The dataframe is a lot simpler and easier to understand now and can be used to easily identify teams.

Before finalizing the teams dataframe, lets ensure there are no missing values.

In [8]:
dfTeamsUC2.isna().sum()

market            0
alias             0
name              0
id                0
conf_name         0
conf_alias        0
conf_id           0
venue_capacity    0
dtype: int64

Since there are no missing values, we can move on to the next csv file

In [9]:
dfTeams = dfTeamsUC2.copy()

In [10]:
dfTeams.to_csv('teams_clean.csv', index=False)

### teams_box_scores.csv

The teams_box_scores.csv data is going to be used to cluster teams. To clean the data 

In [11]:
# load the data, the warning that results is due to NaN values
dfTeamBoxUC = pd.read_csv('team_box_scores.csv')
dfTeamBoxUC.head()

/var/folders/5s/_hqjmj3d7xgb3w8_skgs580r0000gn/T/ipykernel_1715/1520690117.py:2: DtypeWarning: Columns (4,7,8,9,10,11,16,21,58) have mixed types. Specify dtype option on import or set low_memory=False.
  dfTeamBoxUC = pd.read_csv('team_box_scores.csv')


,game_id,season,status,coverage,neutral_site,scheduled_date,gametime,conference_game,tournament,tournament_type,...,opp_fast_break_pts,opp_second_chance_pts,opp_team_turnovers,opp_points_off_turnovers,opp_team_rebounds,opp_flagrant_fouls,opp_player_tech_fouls,opp_team_tech_fouls,opp_coach_tech_fouls,created
0,4069f80e-04f0-4f69-a563-86014bbe95a0,2015,closed,full,NaN,2015-12-03,2015-12-03 03:00:00 UTC,NaN,NaN,NaN,...,0.0,3.0,0.0,10.0,4.0,0.0,0.0,0.0,0.0,2018-02-20 15:48:54 UTC
1,7160a0e0-bbc3-46ad-afc6-e4e6b5b90a51,2015,closed,full,NaN,2016-01-17,2016-01-17 02:00:00 UTC,NaN,NaN,NaN,...,0.0,0.0,1.0,5.0,4.0,0.0,0.0,1.0,0.0,2018-02-20 15:48:54 UTC
2,320ccf7a-8a32-4ce6-a561-10687985c6a6,2015,closed,full,NaN,2015-12-22,2015-12-22 20:00:00 UTC,NaN,NaN,NaN,...,2.0,8.0,1.0,6.0,1.0,0.0,0.0,0.0,0.0,2018-02-20 15:48:53 UTC
3,4069f80e-04f0-4f69-a563-86014bbe95a0,2015,closed,full,NaN,2015-12-03,2015-12-03 03:00:00 UTC,NaN,NaN,NaN,...,0.0,17.0,0.0,31.0,7.0,0.0,0.0,0.0,0.0,2018-02-20 15:48:54 UTC
4,7160a0e0-bbc3-46ad-afc6-e4e6b5b90a51,2015,closed,full,NaN,2016-01-17,2016-01-17 02:00:00 UTC,NaN,NaN,NaN,...,21.0,16.0,1.0,30.0,3.0,0.0,0.0,0.0,0.0,2018-02-20 15:48:54 UTC


In [12]:
# list all of the columns
', '.join(dfTeamBoxUC.columns)

'game_id, season, status, coverage, neutral_site, scheduled_date, gametime, conference_game, tournament, tournament_type, tournament_round, tournament_game_no, attendance, lead_changes, times_tied, periods, possession_arrow, venue_id, venue_city, venue_state, venue_address, venue_zip, venue_country, venue_name, venue_capacity, home_team, name, market, team_id, alias, league_id, league_name, league_alias, conf_id, conf_name, conf_alias, division_id, division_name, division_alias, logo_large, logo_medium, logo_small, opp_name, opp_market, opp_id, opp_alias, opp_league_id, opp_league_name, opp_league_alias, opp_conf_id, opp_conf_name, opp_conf_alias, opp_division_id, opp_division_name, opp_division_alias, opp_logo_large, opp_logo_medium, opp_logo_small, win, points_game, minutes, field_goals_made, field_goals_att, field_goals_pct, three_points_made, three_points_att, three_points_pct, two_points_made, two_points_att, two_points_pct, blocked_att, free_throws_made, free_throws_att, free_thr

As you can see, a lot of columns here are repeats from the above teams dataframe. We will remove those columns now.

In [13]:
dfTeamBoxUC1 = dfTeamBoxUC.drop(['venue_id', 'venue_city', 'venue_state', 'venue_address', 
                                 'venue_zip', 'venue_country', 'venue_name',
                                 'name', 'market', 'alias', 'league_id', 'league_name', 'league_alias', 
                                 'conf_name', 'conf_alias', 'division_id', 'division_name', 'division_alias', 
                                 'logo_large', 'logo_medium', 'logo_small', 'opp_name', 'opp_market', 'opp_alias', 
                                 'opp_league_id', 'opp_league_name', 'opp_league_alias', 'opp_conf_name', 
                                 'opp_conf_alias', 'opp_division_id', 'opp_division_name', 'opp_division_alias', 
                                 'opp_logo_large', 'opp_logo_medium', 'opp_logo_small'], axis=1)

We keep the team_id, conf_id, opp_id, and opp_conf_id so that we can link the teams in the game to corresponding teams in the team dataframe. We keep the conference ids for the same reason.

Let's look at the remaining columns

In [14]:
# list all of the columns
', '.join(dfTeamBoxUC1.columns)

'game_id, season, status, coverage, neutral_site, scheduled_date, gametime, conference_game, tournament, tournament_type, tournament_round, tournament_game_no, attendance, lead_changes, times_tied, periods, possession_arrow, venue_capacity, home_team, team_id, conf_id, opp_id, opp_conf_id, win, points_game, minutes, field_goals_made, field_goals_att, field_goals_pct, three_points_made, three_points_att, three_points_pct, two_points_made, two_points_att, two_points_pct, blocked_att, free_throws_made, free_throws_att, free_throws_pct, offensive_rebounds, defensive_rebounds, rebounds, assists, turnovers, steals, blocks, assists_turnover_ratio, personal_fouls, ejections, foulouts, points, fast_break_pts, second_chance_pts, team_turnovers, points_off_turnovers, team_rebounds, flagrant_fouls, player_tech_fouls, team_tech_fouls, coach_tech_fouls, opp_points_game, opp_minutes, opp_field_goals_made, opp_field_goals_att, opp_field_goals_pct, opp_three_points_made, opp_three_points_att, opp_three

We want to keep all columns after and including the win column as these are boolean or numerical statistics regarding the game. We do not need the final created column however as this is a timestamp that does not relate to our analysis.

The only columns left to look at are the following:

game_id, season, status, coverage, neutral_site, scheduled_date, gametime, conference_game, tournament, tournament_type, tournament_round, tournament_game_no, attendance, lead_changes, times_tied, periods, possession_arrow, home_team

We will want to keep game_id, season, attendance, lead_changes, times_tied, periods, and home team but the rest can be discarded as they will not be useful for our purposes.

In [15]:
dfTeamBoxUC2 = dfTeamBoxUC1.drop(['status', 'coverage', 'neutral_site', 'scheduled_date', 'gametime', 
'conference_game', 'tournament', 'tournament_type', 'tournament_round', 
'tournament_game_no', 'possession_arrow', 'created'], axis=1)

At this point, all the columns in our dataframe dfTeamBoxUC2 could be interesting for clustering teams. Now we need to take care of any missing data.

In [16]:
# look at 30 columns with the least number of missing values, excluding the columns with no missing values
dfTeamBoxUC2.isna().sum()[lambda x: x > 0].sort_values(ascending=True)[:30]

win                              2
periods                         62
conf_id                         72
opp_conf_id                     72
venue_capacity                 294
points                        8019
opp_two_points_made           8019
opp_three_points_pct          8019
opp_three_points_att          8019
opp_three_points_made         8019
opp_field_goals_pct           8019
opp_minutes                   8019
opp_field_goals_made          8019
opp_two_points_att            8019
player_tech_fouls             8019
flagrant_fouls                8019
opp_field_goals_att           8019
opp_two_points_pct            8019
opp_free_throws_att           8019
opp_free_throws_made          8019
opp_personal_fouls            8019
opp_free_throws_pct           8019
opp_offensive_rebounds        8019
opp_defensive_rebounds        8019
opp_rebounds                  8019
opp_assists                   8019
opp_turnovers                 8019
opp_steals                    8019
opp_blocks          

It appears that there are 8019 rows that have missing values in a lot of columns. Let's get rid of those rows and then see what remains.

In [17]:
dfTeamBoxUC3 = dfTeamBoxUC2.dropna(subset=['opp_free_throws_att', 'assists'])

Now lets look at the columns with missing data again

In [18]:
dfTeamBoxUC3.isna().sum()[lambda x: x > 0].sort_values(ascending=True)

win                             2
periods                        62
conf_id                        66
opp_conf_id                    66
venue_capacity                240
lead_changes                  656
times_tied                    728
opp_team_rebounds             993
team_rebounds                 993
opp_points_off_turnovers     1336
points_off_turnovers         1336
second_chance_pts            1361
opp_second_chance_pts        1361
team_turnovers               1502
opp_team_turnovers           1502
attendance                   2522
opp_fast_break_pts           2970
fast_break_pts               2970
opp_ejections               15532
ejections                   15532
opp_team_tech_fouls         27491
team_tech_fouls             27491
opp_foulouts                27522
foulouts                    27522
coach_tech_fouls            29142
opp_coach_tech_fouls        29142
dtype: int64

It seems the following columns have a lot of missing data: 

ejections, opp_ejections, opp_foulouts, foulouts, team_tech_fouls, opp_team_tech_fouls, coach_tech_fouls, opp_coach_tech_fouls

Since these statistics are relatively unique and not extremely important for analyzing how teams play, we can drop them. 



In [19]:
dfTeamBoxUC4 = dfTeamBoxUC3.drop(['ejections', 'opp_ejections', 'opp_foulouts', 'foulouts', 'team_tech_fouls', 
                                           'opp_team_tech_fouls', 'coach_tech_fouls', 'opp_coach_tech_fouls'], axis=1)

Now let's look at the remaining missing data one more time.

In [20]:
dfTeamBoxUC4.isna().sum()[lambda x: x > 0].sort_values(ascending=True)

win                            2
periods                       62
conf_id                       66
opp_conf_id                   66
venue_capacity               240
lead_changes                 656
times_tied                   728
opp_team_rebounds            993
team_rebounds                993
opp_points_off_turnovers    1336
points_off_turnovers        1336
second_chance_pts           1361
opp_second_chance_pts       1361
team_turnovers              1502
opp_team_turnovers          1502
attendance                  2522
opp_fast_break_pts          2970
fast_break_pts              2970
dtype: int64

At this point I want to try and fill in the missing data instead of dropping rows. I will go through the columns now.

#### Wins

In [21]:
dfTeamBoxUC4[dfTeamBoxUC4['win'].isna()][['points_game', 'opp_points_game', 'points', 'opp_points', 'win']]

,points_game,opp_points_game,points,opp_points,win
27540,0,0,98.0,84.0,NaN
33556,0,0,84.0,98.0,NaN


It appears that the win column isn't populated because the points_game and opp_points_game columns are empty. It also appears that points_game and opp_points_game might be duplicate columns of points and opp_points. Let's fix these two rows and then see if we can drop two of these duplicated columns.

In [22]:
# fix missing win columns
missing_win_rows_index = dfTeamBoxUC4[dfTeamBoxUC4['win'].isna()].index
dfTeamBoxUC4.loc[missing_win_rows_index, 'points_game'] = dfTeamBoxUC4.loc[missing_win_rows_index, 'points']
dfTeamBoxUC4.loc[missing_win_rows_index, 'opp_points_game'] = dfTeamBoxUC4.loc[missing_win_rows_index, 'opp_points']
dfTeamBoxUC4.loc[missing_win_rows_index, 'win'] = dfTeamBoxUC4.loc[missing_win_rows_index, 'points_game'] > dfTeamBoxUC4.loc[missing_win_rows_index, 'opp_points_game']

In [23]:
# see if we can get rid of points_game or points
dfTeamBoxUC4[dfTeamBoxUC4['opp_points_game'] != dfTeamBoxUC4['opp_points'].astype(int)][['season', 'team_id', 'opp_id', 'points_game', 'opp_points_game', 'points', 'opp_points', 'win']]

,season,team_id,opp_id,points_game,opp_points_game,points,opp_points,win
41,2014,620d5944-7156-47d3-aad5-5b3824557d03,441a11b4-b506-45b3-8030-fe72a4381c40,61,70,61.0,72.0,False
488,2014,fa416692-7e09-4f0a-9bcf-0cf7d5149a14,b795ddbc-baab-4499-8803-52e8608520ab,68,81,136.0,162.0,False
550,2014,a52b2ece-1f87-45b5-ae1e-8d0920479965,1f99a164-d593-4d81-85d5-0d7889d6f486,59,71,59.0,74.0,False
2634,2013,55af9c69-3675-4b9b-a882-96640b782fff,054c3e85-0552-4549-b123-7e84af6e7b6c,79,78,79.0,81.0,True
5920,2014,b795ddbc-baab-4499-8803-52e8608520ab,fa416692-7e09-4f0a-9bcf-0cf7d5149a14,81,68,162.0,136.0,True
...,...,...,...,...,...,...,...,...
56798,2016,a52b2ece-1f87-45b5-ae1e-8d0920479965,e57bd0ba-2f36-48a8-ab40-c262edb4d857,73,59,73.0,0.0,True
56866,2017,3b6293f3-03db-46d2-8793-248963c2d83b,d4dc55b3-94fe-4d4f-b0bf-c50c5f064392,76,77,76.0,90.0,False
58640,2017,4aebd148-8119-4875-954c-66779867989b,24051034-96bb-4f78-a3a6-312f3258780f,85,54,85.0,56.0,True
59249,2017,15d31915-fbd6-4ae3-8e4b-f3b563c56a18,6dfaf0ba-47c4-4e05-b0a7-72734747d48f,75,83,75.0,82.0,False


In [24]:
dfTeams[dfTeams['id'] == '620d5944-7156-47d3-aad5-5b3824557d03']

,market,alias,name,id,conf_name,conf_alias,conf_id,venue_capacity
159,San Diego,USD,Toreros,620d5944-7156-47d3-aad5-5b3824557d03,West Coast,WCC,c664ceee-1dc0-4743-a6d8-11fbdfb87f61,5100


In [25]:
dfTeams[dfTeams['id'] == '441a11b4-b506-45b3-8030-fe72a4381c40']

,market,alias,name,id,conf_name,conf_alias,conf_id,venue_capacity
150,Loyola Marymount,LMU,Lions,441a11b4-b506-45b3-8030-fe72a4381c40,West Coast,WCC,c664ceee-1dc0-4743-a6d8-11fbdfb87f61,3900


In [26]:
dfTeamBoxUC4[dfTeamBoxUC4['rebounds'] != dfTeamBoxUC4['team_rebounds']][['rebounds', 'team_rebounds']]

,rebounds,team_rebounds
0,37.0,7.0
1,42.0,3.0
2,46.0,4.0
3,34.0,4.0
4,20.0,4.0
...,...,...
59605,29.0,6.0
59606,34.0,4.0
59607,37.0,5.0
59608,35.0,3.0


In [27]:
dfTeamBoxUC4[dfTeamBoxUC4['turnovers'] != dfTeamBoxUC4['team_turnovers']][['turnovers', 'team_turnovers']]

,turnovers,team_turnovers
0,13.0,0.0
1,17.0,1.0
2,11.0,0.0
3,23.0,0.0
4,21.0,1.0
...,...,...
59605,9.0,0.0
59606,16.0,0.0
59607,13.0,0.0
59608,8.0,1.0


It appears that the columns points_game and points are not identical. The same applies for the opponent version of these columns. At first glance points_game and opp_points_game look to be like the correct values. A quick reality check by comparing these values to final scores found on the ESPN website confirms my guess. I will remove the points and opp_points column as they do not seem to have the accurate game scores. It appears that there is a simmilar phenomena for rebounds and turnovers, so I will remove these too.

In [28]:
dfTeamBoxUC5 = dfTeamBoxUC4.drop(['points', 'opp_points', 'team_rebounds', 
                                  'opp_team_rebounds', 'team_turnovers', 'opp_team_turnovers'], axis=1)

#### Periods

We can fill in the periods column by looking at how many minutes were played in the game and entering the corresponding number of periods.

In [29]:
dfTeamBoxUC5[dfTeamBoxUC5['periods'].isna()][['points_game', 'opp_points_game', 'win', 'minutes', 'opp_minutes']]

,points_game,opp_points_game,win,minutes,opp_minutes
42286,59,80,False,3:20:00,3:20:00
42287,80,59,True,3:20:00,3:20:00
42288,66,58,True,3:20:00,3:20:00
42289,58,66,False,3:20:00,3:20:00
42290,86,83,True,3:20:00,3:20:00
...,...,...,...,...,...
58555,69,64,True,3:20:00,3:20:00
58556,60,75,False,3:20:00,3:20:00
58557,75,60,True,3:20:00,3:20:00
58558,66,82,False,3:20:00,3:20:00


3:20:00 seems to be the standard number of minutes in a game, which corresponds to 40 total minutes from 2 periods or halves. Any row with missing period and 3:20:00 in the minutes column can be set to 2.

In [30]:
missing_2_periods_rows_index = dfTeamBoxUC5[(dfTeamBoxUC5['periods'].isna()) & (dfTeamBoxUC5['minutes'] == '3:20:00')].index
dfTeamBoxUC5.loc[missing_2_periods_rows_index, 'periods'] = 2
dfTeamBoxUC5[dfTeamBoxUC5['periods'].isna()][['points_game', 'opp_points_game', 'win', 'minutes', 'opp_minutes']]

,points_game,opp_points_game,win,minutes,opp_minutes
42308,103,98,True,4:10:00,4:10:00
42309,98,103,False,4:10:00,4:10:00
42338,58,67,False,3:45:00,3:45:00
42339,67,58,True,3:45:00,3:45:00


There are still four rows with missing periods. After looking at the data I found these minutes correspond to 3 and 4 periods.

In [31]:
missing_3_periods_rows_index = dfTeamBoxUC5[(dfTeamBoxUC5['periods'].isna()) & (dfTeamBoxUC5['minutes'] == '3:45:00')].index
dfTeamBoxUC5.loc[missing_3_periods_rows_index, 'periods'] = 3
missing_4_periods_rows_index = dfTeamBoxUC5[(dfTeamBoxUC5['periods'].isna()) & (dfTeamBoxUC5['minutes'] == '4:10:00')].index
dfTeamBoxUC5.loc[missing_4_periods_rows_index, 'periods'] = 4
dfTeamBoxUC5[dfTeamBoxUC5['periods'].isna()][['points_game', 'opp_points_game', 'win', 'minutes', 'opp_minutes']]

,points_game,opp_points_game,win,minutes,opp_minutes


Now we have all of the missing period data filled. At this point I also want to reformat the minutes column to be an integer number representing the number of minutes played instead of a string.

In [32]:
dfTeamBoxUC5['minutes'].unique()

array(['3:20:00', '3:45:00', '4:35:00', '4:10:00', '5:00:00'],
      dtype=object)

These unique minute values correspond to 2, 3, 4, 5, and 6 periods being played. The first two periods in basketball are 20 minutes long and then the following periods are overtime periods which are each 5 minutes. Column values should be mapped in the following manner:

3:20:00 -> 40,
3:45:00 -> 45,
4:35:00 -> 50,
4:10:00 -> 55,
5:00:00 -> 60

In [33]:
# reformat minutes column
dfTeamBoxUC5.loc[dfTeamBoxUC5['minutes'] == '3:20:00', 'minutes'] = 40
dfTeamBoxUC5.loc[dfTeamBoxUC5['minutes'] == '3:45:00', 'minutes'] = 45
dfTeamBoxUC5.loc[dfTeamBoxUC5['minutes'] == '4:10:00', 'minutes'] = 50
dfTeamBoxUC5.loc[dfTeamBoxUC5['minutes'] == '4:35:00', 'minutes'] = 55
dfTeamBoxUC5.loc[dfTeamBoxUC5['minutes'] == '5:00:00', 'minutes'] = 60
dfTeamBoxUC5['minutes'] = dfTeamBoxUC5['minutes'].astype(int)
dfTeamBoxUC5['minutes'].unique()

array([40, 45, 55, 50, 60])

We can also drop the opp_minutes column as it is the same as the minutes column.

In [34]:
dfTeamBoxUC5.drop('opp_minutes', axis=1, inplace=True)

#### conf_id and opp_conf_id

At first I thought these values could be filled by getting the conference id of the game's teams from the team dataframe.

However, as I looked at the data I found that some of the games recorded have teams that are not in the teams dataframe. This could include games where D1 teams play lower division teams.

At this point I will get rid of any games that include teams not in our team dataframe and then see what data is still missing.

In [35]:
len(dfTeamBoxUC5[(~dfTeamBoxUC5['team_id'].isin(dfTeams['id'])) | (~dfTeamBoxUC5['opp_id'].isin(dfTeams['id']))])

3580

There are 3,580 games that include teams not in our teams dataframe. Let's remove them.

In [36]:
dfTeamBoxUC6 = dfTeamBoxUC5[(dfTeamBoxUC5['team_id'].isin(dfTeams['id'])) & (dfTeamBoxUC5['opp_id'].isin(dfTeams['id']))]

In [37]:
dfTeamBoxUC6.isna().sum()[lambda x: x > 0].sort_values(ascending=True)

venue_capacity               224
points_off_turnovers         519
opp_points_off_turnovers     519
lead_changes                 526
second_chance_pts            542
opp_second_chance_pts        542
times_tied                   572
fast_break_pts              2042
opp_fast_break_pts          2042
attendance                  2378
dtype: int64

Removing those games also got rid of our rows with missing conference data.

For the remaining missing data columns, except lead_changes, times_tied, and attendance, I am going to fill the missing values with the average value for the statistics corresponding team.

#### fast_break_pts, second_chance_pts, points_off_turnovers

In [38]:
for col in ['fast_break_pts', 'second_chance_pts', 'points_off_turnovers']:
    avg_team_stat = dfTeamBoxUC6.groupby('team_id')[col].mean().round()
    # fill in team state
    for team_id, avg in avg_team_stat.items():
        dfTeamBoxUC6.loc[dfTeamBoxUC6['team_id'] == team_id, col] = dfTeamBoxUC6.loc[dfTeamBoxUC6['team_id'] == team_id, col].fillna(avg)
    # fill in opp team state
    for team_id, avg in avg_team_stat.items():
        dfTeamBoxUC6.loc[dfTeamBoxUC6['opp_id'] == team_id, 'opp_' + col] = dfTeamBoxUC6.loc[dfTeamBoxUC6['opp_id'] == team_id, 'opp_' + col].fillna(avg)

#### attendance and venue_capacity

First, I will get rid of any rows where both attendance and venue capacity are empty. Then, I will fill venue capacity and attendance using the average percentage attendance of all games.

In [39]:
dfTeamBoxUC7 = dfTeamBoxUC6[(~dfTeamBoxUC6['attendance'].isna()) | (~dfTeamBoxUC6['venue_capacity'].isna())]

In [40]:
avg_percent_attendance = (dfTeamBoxUC7['attendance'] / dfTeamBoxUC7['venue_capacity']).mean()
dfTeamBoxUC7.loc[dfTeamBoxUC7['attendance'].isna(), 'attendance'] = (dfTeamBoxUC7.loc[dfTeamBoxUC7['attendance'].isna(), 'venue_capacity'] * avg_percent_attendance).round()
dfTeamBoxUC7.loc[dfTeamBoxUC7['venue_capacity'].isna(), 'venue_capacity'] = (dfTeamBoxUC7.loc[dfTeamBoxUC7['venue_capacity'].isna(), 'attendance'] * (1/ avg_percent_attendance)).round()

#### lead_changes and times_tied

lead_changes and times_tied will be filled in with the average lead changes and times tied of one of the teams in each game.

In [41]:
avg_lead_changes = dfTeamBoxUC7.groupby('team_id')['lead_changes'].mean().round()
avg_times_tied = dfTeamBoxUC7.groupby('team_id')['times_tied'].mean()
# fill lead changes
for game_id in dfTeamBoxUC7[dfTeamBoxUC7['lead_changes'].isna()]['game_id'].unique():
    find_condition = (dfTeamBoxUC7['lead_changes'].isna()) & (dfTeamBoxUC7['game_id'] == game_id)
    dfTeamBoxUC7.loc[find_condition, 'lead_changes'] = avg_lead_changes[dfTeamBoxUC7.loc[find_condition, 'team_id'].values[0]]
# fill times tied
for game_id in dfTeamBoxUC7[dfTeamBoxUC7['times_tied'].isna()]['game_id'].unique():
    find_condition = (dfTeamBoxUC7['times_tied'].isna()) & (dfTeamBoxUC7['game_id'] == game_id)
    dfTeamBoxUC7.loc[find_condition, 'times_tied'] = avg_times_tied[dfTeamBoxUC7.loc[find_condition, 'team_id'].values[0]]

In [42]:
dfTeamBoxUC7.isna().sum()[lambda x: x > 0].sort_values(ascending=True)

Series([], dtype: int64)

As you can see there are no more missing values. The team box score data has been effectively cleaned. 

In [43]:
dfTeamBoxUC7.columns

Index(['game_id', 'season', 'attendance', 'lead_changes', 'times_tied',
       'periods', 'venue_capacity', 'home_team', 'team_id', 'conf_id',
       'opp_id', 'opp_conf_id', 'win', 'points_game', 'minutes',
       'field_goals_made', 'field_goals_att', 'field_goals_pct',
       'three_points_made', 'three_points_att', 'three_points_pct',
       'two_points_made', 'two_points_att', 'two_points_pct', 'blocked_att',
       'free_throws_made', 'free_throws_att', 'free_throws_pct',
       'offensive_rebounds', 'defensive_rebounds', 'rebounds', 'assists',
       'turnovers', 'steals', 'blocks', 'assists_turnover_ratio',
       'personal_fouls', 'fast_break_pts', 'second_chance_pts',
       'points_off_turnovers', 'flagrant_fouls', 'player_tech_fouls',
       'opp_points_game', 'opp_field_goals_made', 'opp_field_goals_att',
       'opp_field_goals_pct', 'opp_three_points_made', 'opp_three_points_att',
       'opp_three_points_pct', 'opp_two_points_made', 'opp_two_points_att',
       'opp_two

In [44]:
dfTeamBox = dfTeamBoxUC7.copy()

In [45]:
dfTeamBox.reset_index(drop=True, inplace=True)

In [46]:
dfTeamBox.to_csv('team_box_scores_clean.csv', index=False)

### player_box_scores.csv

The player_box_scores.csv data is going to be used to cluster players and possibly teams.

In [47]:
# load the data
dfPlayerBoxUC = pd.read_csv('player_box_scores.csv')
dfPlayerBoxUC.head()

/var/folders/5s/_hqjmj3d7xgb3w8_skgs580r0000gn/T/ipykernel_1715/1831043014.py:2: DtypeWarning: Columns (21,22,39) have mixed types. Specify dtype option on import or set low_memory=False.
  dfPlayerBoxUC = pd.read_csv('player_box_scores.csv')


,game_id,season,neutral_site,scheduled_date,gametime,tournament,tournament_type,tournament_round,tournament_game_no,player_id,...,assists,turnovers,steals,blocks,assists_turnover_ratio,personal_fouls,tech_fouls,flagrant_fouls,points,sp_created
0,14ab9c26-b586-4f68-8989-f433bb3a3e7f,2017,False,2017-11-22,2017-11-22 00:00:00 UTC,NaN,NaN,NaN,NaN,b8df0122-7f1e-4189-a47b-1e08050bf6c6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-20 13:03:24 UTC
1,64667cdd-9379-4ecc-877a-3fb4d76fbff2,2017,False,2017-12-19,2017-12-19 01:00:00 UTC,NaN,NaN,NaN,NaN,2036297a-f0e1-4d65-8cc2-94d3d4de314f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-20 13:03:27 UTC
2,fcc2decd-b14f-4fed-8a78-8856c6689c74,2017,False,2017-11-12,2017-11-12 18:00:00 UTC,NaN,NaN,NaN,NaN,c367fd91-183a-4d7d-9307-b8a289cb7bc9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-20 13:03:23 UTC
3,7cda8dca-e87b-4b4e-9eed-68cfba948957,2017,False,2017-11-26,2017-11-26 21:30:00 UTC,NaN,NaN,NaN,NaN,0ca3800d-fb89-4151-90c0-a3e03835fd04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-20 13:03:26 UTC
4,b3b15b00-c5a6-4239-9c99-713ebe01b8c7,2017,False,2018-01-24,2018-01-24 01:00:00 UTC,NaN,NaN,NaN,NaN,269dc916-2208-40e2-a5ee-062a38a3a00d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-02-20 13:03:21 UTC


In [48]:
# list all of the columns
dfPlayerBoxUC.columns

Index(['game_id', 'season', 'neutral_site', 'scheduled_date', 'gametime',
       'tournament', 'tournament_type', 'tournament_round',
       'tournament_game_no', 'player_id', 'last_name', 'first_name',
       'full_name', 'abbr_name', 'status', 'jersey_number', 'height', 'weight',
       'birth_place', 'birthplace_city', 'birthplace_state',
       'birthplace_country', 'class', 'team_name', 'team_market', 'team_id',
       'team_alias', 'conf_name', 'conf_alias', 'division_name',
       'division_alias', 'league_name', 'home_team', 'active', 'played',
       'starter', 'minutes', 'minutes_int64', 'position', 'primary_position',
       'field_goals_made', 'field_goals_att', 'field_goals_pct',
       'three_points_made', 'three_points_att', 'three_points_pct',
       'two_points_made', 'two_points_att', 'two_points_pct', 'blocked_att',
       'free_throws_made', 'free_throws_att', 'free_throws_pct',
       'offensive_rebounds', 'defensive_rebounds', 'rebounds', 'assists',
       'turnov

Based on the clustering we will be doing, the following columns will not be useful and can be removed:
'neutral_site', 'scheduled_date', 'gametime',
       'tournament', 'tournament_type', 'tournament_round',
       'tournament_game_no', 'last_name', 'first_name',
       'abbr_name', 'status', 'jersey_number', 'birth_place', 'birthplace_city', 
       'birthplace_country', 'team_name', 'team_market',
       'team_alias', 'conf_name', 'conf_alias', 'division_name',
       'division_alias', 'league_name', 'minutes', 'primary_position',
       'sp_created'

In [49]:
dfPlayerBoxUC1 = dfPlayerBoxUC.drop(['neutral_site', 'scheduled_date', 'gametime',
       'tournament', 'tournament_type', 'tournament_round',
       'tournament_game_no', 'last_name', 'first_name',
       'abbr_name', 'status', 'jersey_number', 'birth_place', 'birthplace_city', 
       'birthplace_country', 'team_name', 'team_market',
       'team_alias', 'conf_name', 'conf_alias', 'division_name',
       'division_alias', 'league_name', 'minutes', 'primary_position',
       'sp_created'], axis=1)

In [50]:
# remaining columns
dfPlayerBoxUC1.columns

Index(['game_id', 'season', 'player_id', 'full_name', 'height', 'weight',
       'birthplace_state', 'class', 'team_id', 'home_team', 'active', 'played',
       'starter', 'minutes_int64', 'position', 'field_goals_made',
       'field_goals_att', 'field_goals_pct', 'three_points_made',
       'three_points_att', 'three_points_pct', 'two_points_made',
       'two_points_att', 'two_points_pct', 'blocked_att', 'free_throws_made',
       'free_throws_att', 'free_throws_pct', 'offensive_rebounds',
       'defensive_rebounds', 'rebounds', 'assists', 'turnovers', 'steals',
       'blocks', 'assists_turnover_ratio', 'personal_fouls', 'tech_fouls',
       'flagrant_fouls', 'points'],
      dtype='object')

Before we look at what data is missing, let's remove any box score data from games that aren't in are final cleaned dfTeamBox dataset.

In [51]:
dfPlayerBoxUC2 = dfPlayerBoxUC1[dfPlayerBoxUC1['game_id'].isin(dfTeamBox['game_id'].unique())]

Now lets handle the missing data

In [52]:
dfPlayerBoxUC2.isna().sum()[lambda x: x > 0].sort_values(ascending=True)

height                       433
weight                       433
position                     639
birthplace_state           68936
assists_turnover_ratio    128864
rebounds                  128864
free_throws_pct           128864
two_points_pct            128864
field_goals_pct           128864
three_points_pct          128864
two_points_made           128866
two_points_att            128869
points                    132519
offensive_rebounds        132525
assists                   132560
blocks                    132826
personal_fouls            132829
field_goals_att           132831
defensive_rebounds        132831
three_points_att          132831
steals                    132831
turnovers                 132833
free_throws_att           132839
free_throws_made          132840
three_points_made         132841
field_goals_made          132905
minutes_int64             179774
tech_fouls                197287
blocked_att               393860
flagrant_fouls            393889
class     

It looks like there are some rows that have missing data in a lot of columns, let's try and get rid of those first.

In [53]:
dfPlayerBoxUC3 = dfPlayerBoxUC2.dropna(subset=['field_goals_made'])

In [54]:
dfPlayerBoxUC3.isna().sum()[lambda x: x > 0].sort_values(ascending=True)

two_points_made            2
two_points_att             5
field_goals_att           20
three_points_att          20
three_points_made         21
free_throws_att           22
blocks                    23
assists                   23
free_throws_made          23
offensive_rebounds        23
points                    25
steals                    25
personal_fouls            28
turnovers                 28
defensive_rebounds        34
height                   331
weight                   331
position                 389
minutes_int64          49712
birthplace_state       56497
tech_fouls             64475
blocked_att           260955
flagrant_fouls        260984
class                 463925
dtype: int64

For the missing values in columns up to height, let's fill the columns with the player's average

In [55]:
for col in ['two_points_made', 'two_points_att', 'field_goals_att', 'three_points_att', 'three_points_made', 
'free_throws_att', 'blocks', 'assists', 'free_throws_made', 'offensive_rebounds', 'points', 
'steals', 'personal_fouls', 'turnovers', 'defensive_rebounds']:
    avg_player_stats = dfPlayerBoxUC3.groupby('player_id')[col].mean().round()
    players_to_fill = dfPlayerBoxUC3[dfPlayerBoxUC3[col].isna()]['player_id'].unique()
    # fill in player stat
    for player_id in players_to_fill:
        avg = avg_player_stats[player_id]
        dfPlayerBoxUC3.loc[dfPlayerBoxUC3['player_id'] == player_id, col] = dfPlayerBoxUC3.loc[dfPlayerBoxUC3['player_id'] == player_id, col].fillna(avg)

In [56]:
dfPlayerBoxUC3.isna().sum()[lambda x: x > 0].sort_values(ascending=True)

two_points_att           1
height                 331
weight                 331
position               389
minutes_int64        49712
birthplace_state     56497
tech_fouls           64475
blocked_att         260955
flagrant_fouls      260984
class               463925
dtype: int64

Drop last two_points_att row

In [57]:
dfPlayerBoxUC4 = dfPlayerBoxUC3.dropna(subset=['two_points_att'])

In [58]:
dfPlayerBoxUC4['tech_fouls'].value_counts()

0.0     528698
1.0       4225
2.0         84
3.0         11
4.0          8
5.0          2
6.0          2
9.0          1
23.0         1
14.0         1
7.0          1
Name: tech_fouls, dtype: int64

In [59]:
dfPlayerBoxUC4['flagrant_fouls'].value_counts()

0.0    335647
1.0       877
2.0         1
Name: flagrant_fouls, dtype: int64

In [60]:
dfPlayerBoxUC4['blocked_att'].value_counts()

0.0    275309
1.0     45795
2.0     11903
3.0      2758
4.0       612
5.0       147
6.0        17
7.0        11
8.0         2
Name: blocked_att, dtype: int64

I am going to drop the class column due to the fact that is it mostly nan values. For the 'flagrant_fouls', 'blocked_att', and 'tech_fouls' columns I will fill nan with 0 as this is the most common value for these stats.

In [61]:
dfPlayerBoxUC5 = dfPlayerBoxUC4.drop(['class'], axis=1)

In [62]:
dfPlayerBoxUC5[['flagrant_fouls', 'blocked_att', 'tech_fouls']] = dfPlayerBoxUC5[['flagrant_fouls', 'blocked_att', 'tech_fouls']].fillna(0)


In [63]:
dfPlayerBoxUC5.isna().sum()[lambda x: x > 0].sort_values(ascending=True)

height                331
weight                331
position              389
minutes_int64       49712
birthplace_state    56496
dtype: int64

I am going to drop the rows that have missing, height, weight, or position data.

In [64]:
dfPlayerBoxUC5.dropna(subset=['height', 'weight', 'position'], inplace=True)

I am going to fill the missing birthplace_state with 'UKN' for unknown

In [65]:
dfPlayerBoxUC5['birthplace_state'] = dfPlayerBoxUC5['birthplace_state'].fillna('UKN')

I am going to fill the missing minutes_int64 data with the mean

In [66]:
minutes_mean = dfPlayerBoxUC5['minutes_int64'].mean().round()
dfPlayerBoxUC5['minutes_int64'] = dfPlayerBoxUC5['minutes_int64'].fillna(minutes_mean)

In [67]:
dfPlayerBoxUC5.isna().sum()[lambda x: x > 0].sort_values(ascending=True)

Series([], dtype: int64)

There is no more missing data and we have narrowed down the columns to only the ones we want, so the dfPlayerBoxUC data is clean.

In [68]:
dfPlayerBox = dfPlayerBoxUC5.copy()

In [69]:
dfPlayerBox.reset_index(drop=True, inplace=True)

In [70]:
dfPlayerBox.to_csv('player_box_scores_clean.csv', index=False)